In [13]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from torchvision.transforms import ToTensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import MNIST
from skimage.transform import resize
import imageio
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
import csv

In [14]:
batchsize = 100
inputsize = 28*84
intermediate_nodes1 = 400

In [15]:
class ShauryaDataset(Dataset):
    def __init__(self,root,subt_train,transform=ToTensor()):
        if(subt_train != '.DS_Store'):
            self.root = root
            self.transform = transform
            self.data = subt_train
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = root + self.data[idx]
        image = Image.open(image_path)
        image = ToTensor()(image).squeeze(0)
        image = resize(image,(28,28*3),mode = 'constant')
        img1 = image[:,:28]
        img2 = image[:,28:56]
        img3 = image[:,56:]
        img1 = img1.reshape(28*28)
        img2 = img2.reshape(28*28)
        img3 = img3.reshape(28*28)
        return image,img1,img2,img3

In [16]:
#mypath = '/Users/rushi/Downloads/SoML-50/data/'
mypath = input("Enter testing data path:")
subt_train = [f for f in listdir(mypath) if isfile(join(mypath, f))]
root = mypath

In [17]:
dataset = ShauryaDataset(root,subt_train)

In [18]:
num_classes_labels = 3
class Net_LABELS(nn.Module):   
    def __init__(self):
        super(Net_LABELS, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear = nn.Sequential(
            nn.Linear(4*6*20,num_classes_labels),
            nn.ReLU(),
        )


    # Defining the forward pass    
    def forward(self, x):
        x = x.reshape(-1,1,28,84)
        x = self.cnn_layers(x)
        x = x.reshape(-1,4*6*20)
        x = self.linear(x)
        return x

    def training_step(self, batch):
        images,img1,img2,img3, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images,img1,img2,img3, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [19]:
new_dataset = DataLoader(dataset,len(dataset))
for batch in new_dataset:
    img,img1,img2,img3 = batch

In [20]:
model_Labels = Net_LABELS()
model_Labels.load_state_dict(torch.load('Last_hope_labels.pt'))
model_Labels.eval()
results_labels = torch.argmax(model_Labels(img),dim=1)

In [22]:
filename = "Apar_wink_wink_1.csv"
fields = []
fields.append("Image_Name")
fields.append("Label")
rows = []
with open(filename,'w',newline = '') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields) 
    for i in range(len(dataset)):
        row = []
        row.append(subt_train[i])
        if(results_labels[i] == 0):
            row.append("prefix")
        elif(results_labels[i] == 1):
            row.append("infix")
        else:
            row.append("postfix")
        rows.append(row)
    csvwriter.writerows(rows)